## Parameters

In [ ]:
# ------- PARAMETERS ------- #

RUNS = [155]   # run indexes

root = '/pnfs/pic.es/data/cta/LST/LST3/Data/ZFITS/' # path to all folders with the .zfits files

# -------------------------- #

# other parameters

dir_files     = 'csv_files/'  # path (relative or absolute) to the .csv files that will be created
only_subrun0  = True          # if we want to extract only the first subrun of a run

## Import packages

In [ ]:
import numpy  as np
import pandas as pd

from traitlets.config import Config
from ctapipe.io       import EventSource
from IPython.display  import clear_output

import auxiliar as aux

LST_camera = aux.find_LST_num(root) # extracting the number of LST

aux.create_folder(dir_files)        # creating the folder if dont exist

# Extract Pedestal and Stdv

In [ ]:
# over all runs
for run in RUNS[:]:

    # finding the run in the folder root
    date, subruns = aux.search(root,run)    

    # to open a second run we need to use the first subrun of the run and
    # extract the timestamp of the first event

    # first subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,
            },
        })  

    path   = root + date + '/'
    path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.' + str(0).zfill(4) + '.fits.fz'
    source = EventSource( input_url = path ,config = config, max_events=1)

    for i, ev in enumerate(source): 
        timestamp = int(ev.trigger.time.value)

    # following subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,

          "EventTimeCalculator": {
            "dragon_module_id": 133 ,
            "dragon_reference_counter": 0,
            "dragon_reference_time": timestamp,
          },
            },
        })
    
    # over all subruns
    if only_subrun0 == True:
        subruns = [0]
    
    # iterate over subruns
    for subrun in subruns[:]:
        
        # directory
        path   = root + date + '/'
        path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.'+str(subrun).zfill(4) + '.fits.fz'
        source = EventSource( input_url = path ,config = config)


        # arrays 
        mean = [[] for i in range(1855)]
        stdv = [[] for i in range(1855)]
        time =  []

        for i, ev in enumerate(source): 

            if i % 1000 == 0:
                print('Run '+str(run)+' Subrun '+str(subrun)+' - '+str(round(100 * i / 53000, 2)) + '%')
                
            for j in range(1855):
                mean[j].append('%.2f' % np.mean(ev.r0.tel[1].waveform[0][j][5:]))   # pedestal
                stdv[j].append('%.2f' % np.std( ev.r0.tel[1].waveform[0][j][5:]))   # stdv
            time.append(ev.trigger.time.value)  
            
        print('Run ' + str(run) + ' Subrun ' + str(subrun) + ' - 100%')
        print('\nCreating the dataframe...')


        # subdividing to have sufficient memory 
        mean1, mean2 = [m[:len(m)//2] for m in mean], [m[len(m)//2:] for m in mean]
        stdv1, stdv2 = [s[:len(s)//2] for s in stdv], [s[len(s)//2:] for s in stdv]        
        time1, time2 = time[:len(time)//2], time[len(time)//2:]
        
        del mean, stdv, time
        
        ############################################
        #we create the matrix for the dataframe
        matrix1 = [time1]  
        names1  = ['Time']
        for i in range(len(mean1)):
            matrix1.append(mean1[i])
            matrix1.append(stdv1[i])
            names1.append('Pixel ' + str(i + 1) + ' mean')
            names1.append('Pixel ' + str(i + 1) + ' stdv')
        
        del mean1, stdv1, time1

        # creating the dataframe using pandas
        data1         = pd.DataFrame(matrix1).T
        data1.columns = names1

        del matrix1 , names1

        # csv document
        data1.to_csv(dir_files + 'data_Run' + str(run) + '_Subrun' + str(subrun) + '_1.csv', index=False)
        
        # clear memory
        del data1
        
        ##################################################
        #we create the matrix for the dataframe
        matrix2 = [time2]  
        names2  = ['Time']
        for i in range(len(mean2)):
            matrix2.append(mean2[i])
            matrix2.append(stdv2[i])
            names2.append('Pixel ' + str(i + 1) + ' mean')
            names2.append('Pixel ' + str(i + 1) + ' stdv')
        
        del mean2, stdv2, time2

        # creating the dataframe using pandas
        data2         = pd.DataFrame(matrix2).T
        data2.columns = names2

        del matrix2 , names2

        # csv document
        data2.to_csv(dir_files + 'data_Run' + str(run) + '_Subrun' + str(subrun) + '_2.csv', index=False)

        # clear memory and console
        clear_output()
        del data2